In [2]:
from boson_data_lib import *
import numpy as np
import scipy.linalg as sl
import os
import pandas as pd
import glob

In [3]:
data_dir = "../DATA/"
tests_dir = "../TESTS/"

In [7]:
best_init_rho =  np.array([[ 0.96693975+0.j        , -0.17832301+0.00560344j],
                               [-0.17832301-0.00560344j,  0.03306025+0.j        ]])

fid_dists = fidelity_distances(data_dir, best_init_rho);

In [4]:
def read_as_dataframe(filename, fid_distances, data_dir, tests_dir):

    df = pd.DataFrame()
    
    print("Processing...")
    
    with h5py.File(tests_dir+filename, "r") as f:
        
        #gammas = ['0.079477', '0.25133', '0.79477', '2.5133', '7.9477', '25.133', '79.477', '251.33']
        gammas = f.keys()
        
        for gamma in gammas:
            
            print(gamma)
            
            g = gamma[6:]
            
            init_states = ["State_D"+str(i) for i in range(1, 21)] #f[gamma].keys()
            
            for state in init_states:
                
                Fidelity = f[gamma][state]["Fidelity"][()]

                ser_len = len(Fidelity)

                gamma_column = [g] * ser_len
                state_column = [state[7:]] * ser_len

                fid_dist_column = [fid_distances[int(state[7:])-1]] * ser_len      

                f_name = data_dir + "/" + state + "_2CUT_data.h5"
                #f_name = data_dir + "\\" + state + "_2CUT_data.h5"
                t, dt = extract_time(f_name, g)
                time_column = t
                gamma_time_column = np.array(t)*float(g) 

                d_ser = {'Gamma': gamma_column,
                         'State': state_column,
                         'Time': time_column, 
                         'gt': gamma_time_column,
                         'Fidelity': Fidelity,
                         'Infidelity': 1-Fidelity,
                         'Distance': fid_dist_column}

                df_ser = pd.DataFrame(data = d_ser)   
                df = pd.concat([df, df_ser])
    
    print(" done!")
    
    return df

In [5]:
#test_file = "POP_LME_6_random_trn4_tst20_2024-Jan-16_at_11-11.h5" #complex Hamiltonian
#test_file = "POP_LME_6_random_trn4_tst20_2024-Jan-16_at_11-29.h5" #simple Hamiltonian
#test_file = "POP_LME_6_tssos_trn4_tst20_2024-Jan-16_at_12-30.h5" #just tssos
#test_file = "POP_LME_6_tssos_div_trn4_tst20_2024-Jan-16_at_12-41.h5" #division
test_file = "POP_LME_6_tssos_scaled_trn4_tst20_2024-Jan-16_at_14-17.h5" #


In [8]:
df = read_as_dataframe("../TESTS/"+test_file, fid_dists, data_dir, tests_dir)

Processing...
gamma_0.079477
gamma_0.25133
gamma_0.79477
gamma_2.5133
gamma_25.133
gamma_251.33
gamma_7.9477
gamma_79.477
 done!


In [9]:
df

,Gamma,State,Time,gt,Fidelity,Infidelity,Distance
0,0.079477,1,0.06000,0.004769,1.000000,-6.039613e-14,0.341925
1,0.079477,1,0.08000,0.006358,0.999992,8.247673e-06,0.341925
2,0.079477,1,0.10000,0.007948,0.999957,4.348099e-05,0.341925
3,0.079477,1,0.12000,0.009537,0.999924,7.649594e-05,0.341925
4,0.079477,1,0.14000,0.011127,0.999933,6.713537e-05,0.341925
...,...,...,...,...,...,...,...
751,79.477,20,0.23800,18.915526,0.999261,7.394958e-04,0.340013
752,79.477,20,0.23825,18.935395,0.999259,7.409953e-04,0.340013
753,79.477,20,0.23850,18.955264,0.999258,7.423752e-04,0.340013
754,79.477,20,0.23875,18.975134,0.999256,7.436328e-04,0.340013


In [10]:
pkl_name = "dataframe_"+test_file[:-3]+".pkl"

df.to_pickle(tests_dir+pkl_name)

In [11]:
pkl_name

'dataframe_POP_LME_6_tssos_scaled_trn4_tst20_2024-Jan-16_at_14-17.pkl'

In [12]:
tests_dir+pkl_name

'../TESTS/dataframe_POP_LME_6_tssos_scaled_trn4_tst20_2024-Jan-16_at_14-17.pkl'